In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import os
import pickle

In [16]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [2]:
# reading the files
infile = open('s01.dat', 'rb')
new_dict = pickle.load(infile, encoding="latin1")
infile.close()
labels=[]
data=[]

for keys in new_dict:
    if(keys=='labels'):
        labels=new_dict[keys]
    else:
        data=new_dict[keys]


#  LABELS 

In [3]:
#------------------------ labels ------------------------#



# The combinations of Valence and Arousal can be converted to emotional states:
    # High Arousal & Positive Valence (Excited, Happy)
    # Low Arousal & Positive Valence (Calm, Relaxed)
    # High Arousal & Negative Valence (Angry, Nervous)
    # Low Arousal & Negative Valence (Sad, Bored)
    
# Arousal can range from inactive (e.g. uninterested, bored) to active (e.g. alert, excited)
# valence ranges from unpleasant (e.g. sad, stressed) to pleasant (e.g. happy, elated)
# Dominance ranges from a helpless and weak feeling (without con- trol) to an empowered feeling (in control of everything).






# valence, arousal, dominance, liking
labels = np.array(labels)

# dealing with LABELS and creating a dataframe
valence = labels[:,0]
arousal = labels[:,1]
dominance = labels[:,2]
liking = labels[:,3]

results = pd.DataFrame({'valence': valence, 'arousal': arousal, 'dominance': dominance, 'liking': liking })
# results.describe()


valenceMedian = 4.5
arousalMedian = 4.5

# Function to check if each trial has positive or negative valence
def positive_valence(trial):
    return "positive" if results.iloc[trial]['valence'] >= valenceMedian else "negative"

# Function to check if each trial has high or low arousal
def high_arousal(trial):
    return "high" if results.iloc[trial]['arousal'] >= arousalMedian else "low"


# Convert ratings to high/low or positive/negative values
labels_encoded = []
for i in range (len(results)):
    labels_encoded.append([positive_valence(i), high_arousal(i)])
labels_encoded = np.reshape(labels_encoded, (40, 2))
labelsCategorized = pd.DataFrame(data=labels_encoded, columns=["valence", "arousal"])


# Dataset with only Valence column
df_valence = labelsCategorized['valence']

# Dataset with only Arousal column
df_arousal = labelsCategorized['arousal']


# High Arousal Positive Valence
highArousal_posValence = results[(results['valence'] >= results['valence'].median()) & (results['arousal'] >= results['arousal'].median())]

# Low Arousal Positive Valence dataset
lowArousal_posValence = results[(results['valence'] >= results['valence'].median()) & (results['arousal'] < results['arousal'].median())]

# High Arousal Negative Valence dataset
highArousal_negValence = results[(results['valence'] < results['valence'].median()) & (results['arousal'] >= results['arousal'].median())]

# Low Arousal Negative Valence dataset
lowArousal_negValence = results[(results['valence'] < results['valence'].median()) & (results['arousal'] < results['arousal'].median())]


# print("High Arousal Positive Valence:", str(len(highArousal_posValence)))
# print("Low Arousal Positive Valence:", str(len(lowArousal_posValence)))
# print("High Arousal Negative Valence:", str(len(highArousal_negValence)))
# print("Low Arousal Negative Valence:", str(len(lowArousal_negValence)))

# DATA

In [4]:
#------------------------ data ------------------------#
all_channels = np.array(['b1','b2', 'b3', 'b4', 'b5', 'b6','b7', 'b8', 'b9', 'b10','b11','b12', 'b13', 'b14', 'b15', 'b16','b17', 'b18', 'b19', 'b20','b21','b22', 'b23', 'b24', 'b25', 'b26','b27', 'b28', 'b29', 'b30', 'b31', 'b32', 'hEOG', 'vEOG', 'zEMG', 'tEMG', 'GSR', 'Respiration belt', 'Plethysmograph', 'Temp'])
peripheral_channels = np.array(["zEMG", "tEMG", "GSR", "Respiration belt", "Plethysmograph", "Temperature"])

# drop unneeded data 
data = data[:, 34:, :]


#convert it to dataframe
df_data = pd.DataFrame(columns = peripheral_channels)
df_dataPerVideo = pd.DataFrame()

for i in range(0, len(data)): #video 40
    for j in range(0, len(data[i])): #channel 8
        temp = []
        for k in range(0, len(data[i][j])): #value 8064
            temp.append(data[i][j][k])
        df_dataPerVideo[peripheral_channels[j]] = temp
    df_data= df_data.append(df_dataPerVideo,ignore_index = True)

# Method 1: taking average

In [6]:
step = 8064;
columnNames=["zEMG", "tEMG", "GSR", "Respiration belt", "Plethysmograph", "Temperature"]
data_avgeraged = pd.DataFrame(columns=columnNames)


# taking average of each video to one value --> each 8064 sample = 1 video 
for i in range(0, len(df_data), step):
    data_avg = df_data.iloc[i:i+8064, :]
    avg = (data_avg.mean(axis=0))
    data_avgeraged= data_avgeraged.append(avg, ignore_index=True)

In [26]:
avgValence = 0;
avgArousal = 0;

for i in range(0,100):
    # TRY FIRST VALENCE
    
    # split the data into train and test
    dataYValence = df_valence
    dataX = data_avgeraged

    x_train, x_test, y_train, y_test = train_test_split(dataX, dataYValence, test_size=0.2, stratify=dataYValence)

    # apply standardization on numerical features
    from sklearn.preprocessing import StandardScaler

    sc = StandardScaler()
    standX_train = sc.fit_transform(x_train)
    standX_test = sc.transform(x_test)

    #Training and Predictions
#     classifier = KNeighborsClassifier(n_neighbors=5)
#     classifier = LogisticRegression()
#     classifier = RandomForestClassifier(n_estimators=100)
#     classifier = GaussianNB()
#     classifier = SVC(kernel='linear')
    classifier = SVC(kernel='rbf')
    classifier.fit(standX_train, y_train)

    #make predictions on our test data
    y_pred = classifier.predict(standX_test)

    #Evaluating the Algorithm
    from sklearn.metrics import accuracy_score
    avgValence = avgValence + accuracy_score(y_test,y_pred)
  


    # THEN TRY AROUSAL

    # split the data into train and test
    dataYArousal = df_arousal
    dataX = data_avgeraged

    x_train, x_test, y_train, y_test = train_test_split(dataX, dataYArousal, test_size=0.2, stratify=dataYArousal)
    x_train = pd.DataFrame(x_train).sort_index()
    y_train = pd.DataFrame(y_train).sort_index()

    # apply standardization on numerical features
    from sklearn.preprocessing import StandardScaler

    sc = StandardScaler()
    standX_train = sc.fit_transform(x_train)
    standX_test = sc.transform(x_test)

    #Training and Predictions
#     classifier = KNeighborsClassifier(n_neighbors=4)
#     classifier = LogisticRegression()
#     classifier = RandomForestClassifier(n_estimators=100)
#     classifier = GaussianNB()
#     classifier =  SVC(kernel='linear')
    classifier = SVC(kernel='rbf')
    classifier.fit(standX_train, y_train)

    #make predictions on our test data
    y_pred = classifier.predict(standX_test)

    #Evaluating the Algorithm
    from sklearn.metrics import accuracy_score
    avgArousal = avgArousal + accuracy_score(y_test,y_pred)
    
    
    
print("Valence accuracy for 100 times: ", avgValence)
print("Arousal accuracy for 100 times: ", avgArousal)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConv

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConv

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConv

Valence accuracy for 100 times:  58.625
Arousal accuracy for 100 times:  59.125


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
